In [1]:
! pip install datasets transformers

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 70 not upgraded.


In [4]:
import transformers

print(transformers.__version__)

4.57.1


# Fine-tuning a language model

In [5]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

In [6]:
datasets["train"][10]

{'text': ' The game \'s battle system , the BliTZ system , is carried over directly from Valkyira Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action Gauge . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,text
0,
1,"The league season concluded with an away match against divisional runners @-@ up Scunthorpe ; having gone two goals down York fought back to draw 2 – 2 with goals scored by Brobbel and Andrew . This result meant York finished the season in seventh @-@ place in League Two , and would thus play fourth @-@ placed Fleetwood in the play @-@ off semi @-@ final on the back of a 17 @-@ match unbeaten run . York lost 1 – 0 to Fleetwood in the first leg at Bootham Crescent ; the goal came from former York player Matty Blair in the 50th @-@ minute , who scored from close range after Antoni Sarcevic 's shot was blocked on the line . A 0 – 0 draw away to Fleetwood in the second leg meant York were eliminated 1 – 0 on aggregate , ending the prospect of a second promotion in three seasons . At an awards night held at York Racecourse , Oyebanjo was voted Clubman of the Year for 2013 – 14 . \n"
2,"The river 's mouth has populations of Canada geese and swans . Other waterfowl include mallards , green @-@ winged teal , and goldeneye . The river supports Bald eagle and osprey populations , whose eyries can be seen high up in black cottonwood and dead conifers . Grouse , especially ruffed grouse , are numerous throughout the valley . \n"
3,= = Reception and home media = = \n
4,The remains of the Byzantine @-@ era church in Jifna testifies to the existence of a Christian community prior to the Muslim conquest . It continued to exist during the Middle Ages and the village is still inhabited mainly by Christians . The names of Christian inhabitants from Jifna appeared in a 10th @-@ century inscription on a stone above the gate of St. George 's Monastery in the Wadi Qelt . \n
5,
6,
7,"Of Thomas Quiney 's fate the records show little . It is speculated that he may have died in 1662 or 1663 when the parish burial records are incomplete . He certainly had a nephew in London , who by this time held the lease to The Cage . \n"
8,"In April 2015 , Activision announced a new entry in the series , titled Guitar Hero Live . The title was developed by Activision 's internal studio FreeStyleGames , who previously had worked on the DJ Hero spinoff titles . FreeStyleGames were given free reign to reboot the Guitar Hero series for next @-@ generation consoles . One of their first innovations was to drop the standard five @-@ button guitar controller , ultimately designing a six @-@ button guitar controller , with two rows of three buttons each , allowing them to mimic actual guitar fingering . Guitar Hero Live was released with both career and an online mode . The career mode used full @-@ motion video taken from the perspective of a lead guitarist underneath the note highway , to create an immersive experience to the player . The online mode , called GHTV , discarded the previous downloadable content approach and used a music video channel approach to stream playable songs to players , adding new songs to the catalog on a weekly basis . The game was released in October 2015 . \n"
9,


In [9]:
model_checkpoint = "distilgpt2"

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [12]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets["train"][1]

{'input_ids': [796, 569, 18354, 7496, 17740, 6711, 796, 220, 198],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
# block_size = tokenizer.model_max_length
block_size = 128

In [15]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/36718 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [17]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Oz'

In [19]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [20]:
from transformers import Trainer, TrainingArguments

In [21]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [23]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shivprakashlps (shivprakashlps-iit-ropar-tif) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.718600,3.636851
2,3.619700,3.617367
3,3.574800,3.613542


TrainOutput(global_step=7002, training_loss=3.665662927445055, metrics={'train_runtime': 832.0977, 'train_samples_per_second': 67.297, 'train_steps_per_second': 8.415, 'total_flos': 1829011929956352.0, 'train_loss': 3.665662927445055, 'epoch': 3.0})

In [24]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 37.10


In [25]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...kitext2/training_args.bin: 100%|##########| 5.91kB / 5.91kB            

  ...kitext2/model.safetensors:   1%|1         | 3.84MB /  328MB            

  ...226367.3f550d51d2f0.833.0:  17%|#7        | 1.66kB / 9.52kB            

  ...227226.3f550d51d2f0.833.1:  17%|#7        |  62.0B /   359B            

CommitInfo(commit_url='https://huggingface.co/wizardoftrap/distilgpt2-finetuned-wikitext2/commit/d1fd038f1f71750400063f6906dee5f0d5d3ad60', commit_message='End of training', commit_description='', oid='d1fd038f1f71750400063f6906dee5f0d5d3ad60', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wizardoftrap/distilgpt2-finetuned-wikitext2', endpoint='https://huggingface.co', repo_type='model', repo_id='wizardoftrap/distilgpt2-finetuned-wikitext2'), pr_revision=None, pr_num=None)